# **Steganography**

In [9]:
import os
from PIL import Image, UnidentifiedImageError # type: ignore
from cryptography.fernet import Fernet # type: ignore
import numpy as np # type: ignore
from skimage.metrics import peak_signal_noise_ratio as psnr, structural_similarity as ssim # type: ignore

### Generate Key untuk Enkripsi

In [3]:
# (jalankan sekali dan simpan kunci)
def generate_key():
    key = Fernet.generate_key()
    with open("secret.key", "wb") as key_file:
        key_file.write(key)

### Load Kunci Enkripsi

In [4]:
def load_key():
    return open("secret.key", "rb").read()

### Convert JPG/JPEG menjadi PNG

In [63]:
def convert_jpgs_in_folder(folder_path):
    # Pastikan folder ada
    if not os.path.isdir(folder_path):
        print("Folder tidak ditemukan!")
        return
    
    # Loop semua file di dalam folder
    for filename in os.listdir(folder_path):
        # Periksa apakah file adalah JPG atau JPEG
        if filename.lower().endswith((".jpg", ".jpeg")):
            file_path = os.path.join(folder_path, filename)  # Path lengkap file
            img = Image.open(file_path).convert("RGB")  # Konversi ke RGB
            
            # Buat path baru dengan ekstensi PNG
            new_file_path = os.path.splitext(file_path)[0] + ".png"
            
            # Simpan sebagai PNG
            img.save(new_file_path, "PNG")

            # Hapus file JPG lama
            os.remove(file_path)

    print("Semua JPG dalam folder telah dikonversi ke PNG.")

In [64]:
convert_jpgs_in_folder("original")

Semua JPG dalam folder telah dikonversi ke PNG.


### Enkripsi dan Dekripsi menggunakan AES

In [65]:
def encrypt_message(message, key):
    cipher = Fernet(key)
    encrypted_message = cipher.encrypt(message.encode())
    return encrypted_message

def decrypt_message(encrypted_message, key):
    cipher = Fernet(key)
    decrypted_message = cipher.decrypt(encrypted_message).decode()
    return decrypted_message

### Menyisipkan Pesan pada Gambar menggunakan LSB Multi-Channel (RGB)

In [66]:

def encode_image(image_path, message, output_path):
    img = Image.open(image_path).convert("RGB")  # Pastikan gambar dalam mode RGB
    pixels = img.load()
    
    key = load_key()
    encrypted_message = encrypt_message(message, key)
    binary_message = ''.join(format(byte, '08b') for byte in encrypted_message) + '00000000'  # Terminator

    data_index = 0
    for y in range(img.height):
        for x in range(img.width):
            if data_index < len(binary_message):
                r, g, b = pixels[x, y]  # Pastikan mode RGB
                new_r = (r & ~1) | int(binary_message[data_index]) if data_index < len(binary_message) else r
                new_g = (g & ~1) | int(binary_message[data_index+1]) if data_index+1 < len(binary_message) else g
                new_b = (b & ~1) | int(binary_message[data_index+2]) if data_index+2 < len(binary_message) else b
                pixels[x, y] = (new_r, new_g, new_b)
                data_index += 3
            else:
                img.save(output_path)
                return
            
def encode_image_for_multiple_data(original_images_dir, message, hidden_images_dir):
    if not os.path.exists(hidden_images_dir):
        os.makedirs(hidden_images_dir)

    for filename in os.listdir(original_images_dir):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
            try:
                original_path = os.path.join(original_images_dir, filename)
                hidden_image_path = os.path.join(hidden_images_dir, filename)
                encode_image(
                    image_path=original_path, 
                    message=message,
                    output_path=hidden_image_path
                    )

            except UnidentifiedImageError:
                print(f"File {filename} bukan gambar yang valid, dilewati.")
            
def decode_image(image_path):
    img = Image.open(image_path).convert("RGB")  # Pastikan mode RGB
    pixels = img.load()

    binary_message = ""

    for y in range(img.height):
        for x in range(img.width):
            r, g, b = pixels[x, y]
            binary_message += str(r & 1) + str(g & 1) + str(b & 1)

    # Konversi biner ke byte
    byte_message = [binary_message[i:i+8] for i in range(0, len(binary_message), 8)]
    
    try:
        # Hilangkan terminator "00000000" jika ada
        extracted_bytes = bytes([int(b, 2) for b in byte_message if b != "00000000"])

        # Load kunci
        key = load_key()

        # Coba lakukan dekripsi
        decrypted_message = decrypt_message(extracted_bytes, key)
        return decrypted_message

    except Exception:
        return "Tidak ada pesan tersembunyi di dalam gambar."


### How to Use

In [67]:
original_images_dir = "original"
hidden_images_dir = "hidden"
message = "Pesan Rahasia!"

In [34]:
# 1️⃣ Jalankan sekali untuk membuat kunci enkripsi:
generate_key()

In [68]:
# 2️⃣ Menyisipkan pesan ke dalam gambar
encode_image_for_multiple_data(original_images_dir, message, hidden_images_dir)

In [73]:
# 3️⃣ Mengekstrak dan mendekripsi pesan dari gambar
message = decode_image(image_path="hidden/99_img.png")
print("Pesan tersembunyi:", message)

Pesan tersembunyi: Pesan Rahasia!


### Evaluasi

In [74]:
def evaluate_image_quality(original_image, encoded_image):
    # Konversi gambar ke array numpy
    original_array = np.array(original_image)
    encoded_array = np.array(encoded_image)
    
    # Pastikan gambar cukup besar untuk dihitung SSIM
    if original_array.shape[0] < 7 or original_array.shape[1] < 7:
        raise ValueError("Gambar terlalu kecil untuk evaluasi SSIM.")
    
    # Evaluasi PSNR dan SSIM
    psnr_value = psnr(original_array, encoded_array)
    
    # Evaluasi SSIM dengan mengatur ukuran jendela dan channel axis
    ssim_value = ssim(original_array, encoded_array, multichannel=True, win_size=3, channel_axis=-1)
    
    return psnr_value, ssim_value

def evaluate_multiple_images(original_dir, hidden_dir):
    psnr_values = []
    ssim_values = []

    for filename in os.listdir(original_dir):
        original_path = os.path.join(original_dir, filename)
        hidden_path = os.path.join(hidden_dir, filename)

        if os.path.isfile(original_path) and os.path.isfile(hidden_path):
            original_img = Image.open(original_path).convert('RGB')
            hidden_img = Image.open(hidden_path).convert('RGB')

            psnr_value, ssim_value = evaluate_image_quality(original_img, hidden_img)

            psnr_values.append(psnr_value)
            ssim_values.append(ssim_value)

    avg_psnr = np.mean(psnr_values)
    avg_ssim = np.mean(ssim_values)
    
    return avg_psnr, avg_ssim

In [75]:
avg_psnr, avg_ssim = evaluate_multiple_images(original_images_dir, hidden_images_dir)

print(f"PSNR: {avg_psnr}")
print(f"SSIM: {avg_ssim}")

PSNR: 85.02709230423208
SSIM: 0.9999982338957922


### Kesimpulan

4️⃣ Penjelasan Teknik yang Digunakan

Enkripsi AES dengan cryptography.Fernet

🔹 Sebelum pesan disisipkan, pesan dienkripsi untuk menambah keamanan.

🔹 Hanya yang memiliki kunci bisa mendekripsi pesan tersebut.

LSB Multi-Channel (RGB)

🔹 Menyisipkan bit pesan ke dalam semua kanal warna (R, G, B) agar lebih banyak data bisa disimpan.

🔹 Menggunakan 00000000 sebagai terminator agar mudah diekstrak.

5️⃣ Kesimpulan

✅ Teknik ini lebih aman karena menggunakan enkripsi AES sebelum penyisipan.

✅ LSB multi-channel meningkatkan kapasitas penyimpanan data dalam gambar.

✅ Pesan yang disisipkan hanya bisa dibaca oleh orang yang memiliki kunci enkripsi.